In [2]:
import os
from pathlib import Path
import pandas as pd

from geopy.distance import geodesic
from shapely.geometry import Point, LineString
import geopandas as gpd

In [3]:
ticket_path = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis"
data_path = os.path.join(ticket_path, "IN3136_EURAC_shared_folder/data_sites")


In [4]:
sites_metadata = pd.read_csv(os.path.join(data_path, "sites_metadata.csv"), index_col=0)
sites_metadata

,country,latitude,longitude,altitude,provider
station_name,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn
carpentras,France,44.08,5.06,100.0,bsrn
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn
magurele_mars,Romania,44.34,26.01,110.0,bsrn
cabauw,Netherlands,51.97,4.93,0.0,bsrn
...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc
gonzaga,Italy,44.96,10.77,16.0,lombardy


In [5]:
ticket_path_fac = "/home/lhn3e/OneDrive-3E/Research/Solar/tickets/2023/IN3046_factors"
coastline_file = os.path.join(ticket_path_fac, 'coastline/ne_10m_coastline.shp')

In [8]:
coastline_data = gpd.read_file(coastline_file)


In [9]:
for index, row in sites_metadata.iterrows():
    site_point = Point(row['longitude'], row['latitude'])
    
    closest_line = coastline_data.geometry.distance(site_point).idxmin()

    closest_line_geometry = coastline_data.geometry.iloc[closest_line]

    closest_point_on_line = closest_line_geometry.interpolate(closest_line_geometry.project(site_point))
    
    distance = geodesic((row['latitude'], row['longitude']), (closest_point_on_line.y, closest_point_on_line.x)).kilometers
    
    sites_metadata.at[index, 'distance_to_coastline_km'] = distance


/tmp/ipykernel_26588/2135657532.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  closest_line = coastline_data.geometry.distance(site_point).idxmin()
/home/lhn3e/envs/lhn_internship/lib/python3.10/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/tmp/ipykernel_26588/2135657532.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  closest_line = coastline_data.geometry.distance(site_point).idxmin()
/home/lhn3e/envs/lhn_internship/lib/python3.10/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/lhn3e/envs/lhn_internship/lib/python3.10/site-

In [10]:
sites_metadata

,country,latitude,longitude,altitude,provider,distance_to_coastline
station_name,,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn,3.211302
carpentras,France,44.08,5.06,100.0,bsrn,57.987479
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn,482.028360
magurele_mars,Romania,44.34,26.01,110.0,bsrn,198.059538
cabauw,Netherlands,51.97,4.93,0.0,bsrn,49.955084
...,...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc,4.175609
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc,357.351298
gonzaga,Italy,44.96,10.77,16.0,lombardy,117.377329
